In [ ]:
import math
import numpy as np
import cupy as cp
# import tensorflow as tf
import cv2
import scipy.sparse as ssp
from scipy import ndimage
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
def interpolate_frames_farneback(frames):
    """
    双方向のオプティカルフローを用いてフレーム補間を行う。

    :param frames: numpy.ndarray, 形状は (m, 128, 128)
    :return: numpy.ndarray, 形状は (2m-1, 128, 128)
    """
    m, h, w = frames.shape
    new_frames = []

    for i in range(m - 1):
        frame1 = frames[i].astype(np.float32)
        frame2 = frames[i + 1].astype(np.float32)

        # オプティカルフローの計算（前方向）
        flow_forward = cv2.calcOpticalFlowFarneback(
            prev=frame1,
            next=frame2,
            flow=None,
            pyr_scale=0.5,
            levels=3,
            winsize=15,
            iterations=3,
            poly_n=5,
            poly_sigma=1.2,
            flags=0
        )

        # オプティカルフローの計算（後方向）
        flow_backward = cv2.calcOpticalFlowFarneback(
            prev=frame2,
            next=frame1,
            flow=None,
            pyr_scale=0.5,
            levels=3,
            winsize=15,
            iterations=3,
            poly_n=5,
            poly_sigma=1.2,
            flags=0
        )

        # フローの半分を使用
        flow_forward_half = flow_forward * 0.5
        flow_backward_half = flow_backward * 0.5

        # 座標の計算
        h_coords, w_coords = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

        # 前のフレームを未来方向にリマップ
        remap_coords_forward = np.stack((w_coords + flow_forward_half[..., 0],
                                         h_coords + flow_forward_half[..., 1]), axis=-1).astype(np.float32)
        frame1_warped = cv2.remap(frame1, remap_coords_forward, None, cv2.INTER_LINEAR)

        # 次のフレームを過去方向にリマップ
        remap_coords_backward = np.stack((w_coords + flow_backward_half[..., 0],
                                          h_coords + flow_backward_half[..., 1]), axis=-1).astype(np.float32)
        frame2_warped = cv2.remap(frame2, remap_coords_backward, None, cv2.INTER_LINEAR)

        # 中間フレームを生成（平均を取る）
        interpolated_frame = 0.5 * (frame1_warped + frame2_warped)

        # フレームの格納
        new_frames.append(interpolated_frame)


    return np.stack(new_frames)


In [ ]:
def interpolate_frames_lk(frames, max_corners=100, quality_level=0.01, min_distance=10):
    """
    Lucas-Kanade法（疎なオプティカルフロー）を用いてフレーム補間を行う。

    :param frames: numpy.ndarray, 形状は (m, 128, 128)
    :param max_corners: int, 特徴点の最大数
    :param quality_level: float, 特徴点の品質レベル（0〜1）
    :param min_distance: int, 特徴点間の最小距離
    :return: numpy.ndarray, 形状は (m-1, 128, 128)
    """
    m, h, w = frames.shape
    new_frames = []

    # 輝度値のスケーリング係数を計算
    # 全フレームの最大値を用いて0〜255にスケーリング
    max_value = np.max(frames)
    if max_value == 0:
        max_value = 1e-6  # ゼロ除算を防ぐ

    scale_factor = 255.0 / max_value

    for i in range(m - 1):
        # フレームのスケーリングと型変換
        frame1 = (frames[i] * scale_factor).astype(np.uint8)
        frame2 = (frames[i + 1] * scale_factor).astype(np.uint8)

        # 特徴点の検出（Shi-Tomasi法）
        prev_points = cv2.goodFeaturesToTrack(
            image=frame1,
            maxCorners=max_corners,
            qualityLevel=quality_level,
            minDistance=min_distance,
            mask=None,
            blockSize=7,
            useHarrisDetector=False
        )

        if prev_points is None:
            interpolated_frame = (frames[i] + frames[i + 1]) / 2.0
            new_frames.append(interpolated_frame)
            continue

        # 特徴点の追跡
        next_points, status, error = cv2.calcOpticalFlowPyrLK(
            prevImg=frame1,
            nextImg=frame2,
            prevPts=prev_points,
            nextPts=None,
            winSize=(15, 15),
            maxLevel=2,
            criteria=(
                cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                10,
                0.03
            )
        )

        # 有効なポイントのみを選択
        status = status.reshape(-1)
        valid_idx = status == 1
        prev_pts_valid = prev_points.reshape(-1, 2)[valid_idx]
        next_pts_valid = next_points.reshape(-1, 2)[valid_idx]

        # 中間の特徴点を計算（線形補間）
        mid_pts = (prev_pts_valid + next_pts_valid) / 2.0

        # 中間フレームの生成
        interpolated_frame = np.zeros_like(frames[i])

        # スケーリング係数を適用して、元の輝度値に戻す
        frame1_orig = frames[i]
        frame2_orig = frames[i + 1]

        # 特徴点の輝度値を線形補間
        for (x0, y0), (x1, y1), (xm, ym) in zip(
            prev_pts_valid, next_pts_valid, mid_pts
        ):
            x0_int, y0_int = int(x0), int(y0)
            x1_int, y1_int = int(x1), int(y1)
            xm_int, ym_int = int(xm), int(ym)

            if 0 <= y0_int < h and 0 <= x0_int < w and 0 <= y1_int < h and 0 <= x1_int < w and 0 <= ym_int < h and 0 <= xm_int < w:
                val0 = frame1_orig[y0_int, x0_int]
                val1 = frame2_orig[y1_int, x1_int]
                valm = (val0 + val1) / 2.0
                interpolated_frame[ym_int, xm_int] = valm

        # 特徴点以外の部分を前後のフレームの平均で補間
        mask = np.zeros_like(frames[i], dtype=bool)
        mask[interpolated_frame > 0] = True
        interpolated_frame[~mask] = (frame1_orig[~mask] + frame2_orig[~mask]) / 2.0

        new_frames.append(interpolated_frame)

    return np.stack(new_frames)

In [ ]:
def matrix_to_tensor(H, m, n):
    return H.reshape(m, m, n, n)


def tensor_to_matrix(tensor, m, n):
    return tensor.reshape(m * m, n * n)


n = 128
_m = 128
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
# DATA_PATH = "../data"
EXP_DATE = "241112"
H_SETTING = "p-5_lmd-100_m-128"
H_mat = np.load(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_{H_SETTING}.npy")
print("H shape:", H_mat.shape, "type(H):", type(H_mat), "H.dtype:", H_mat.dtype)
H_ten = matrix_to_tensor(H_mat, _m, n)

In [ ]:
# フレーム補間
m = 2 * _m - 1  # 255
interpolated_tensor = np.zeros((m, m, n, n), dtype=np.float32)
for i in range(0, m, 2):
    frames = H_ten[i // 2, :, :, :]
    interpolated_frames = interpolate_frames_lk(frames)
    for j in range(m):
        if j % 2 == 0:
            interpolated_tensor[i, j, :, :] = H_ten[i // 2, j // 2, :, :]
        else:
            interpolated_tensor[i, j, :, :] = interpolated_frames[j // 2, :, :]
for j in range(0, m):
    frames = interpolated_tensor[::2, j, :, :]
    interpolated_frames = interpolate_frames_lk(frames)
    for i in range(1, m, 2):
        interpolated_tensor[i, j, :, :] = interpolated_frames[i // 2, :, :]

interpolated_matrix = tensor_to_matrix(interpolated_tensor, m, n)

In [ ]:
# np.save(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy", interpolated_matrix)
print("H shape:", interpolated_matrix.shape, "type(H):", "H.dtype:", interpolated_matrix.dtype)
# print(f"Saved {DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy")

In [ ]:
SAMPLE_NAME = "Cameraman"
sample_image = Image.open(f"{DATA_PATH}/sample_image{n}/{SAMPLE_NAME}.png").convert("L")
sample_image = cp.asarray(sample_image).ravel() / 255

black_img = Image.open(f"{DATA_PATH}/capture_{241107}/Black.png").convert("L")
black = cp.asarray(black_img) / 255

H = cp.asarray(interpolated_matrix)
Hf = H @ sample_image + black.ravel()
Hf = cp.asnumpy(Hf.reshape(m, m))
print("Hf shape:", Hf.shape)

Hf_pil = Image.fromarray((Hf * 255).astype(np.uint8), mode="L")
Hf_pil.save(f"{DATA_PATH}/{EXP_DATE}/Hf_int_lk.png", format='PNG')
display(Hf_pil)

plt.imshow(Hf, cmap='gray', interpolation='nearest')
plt.colorbar()
plt.title('Grayscale Heatmap')
plt.show()

In [ ]:
I = 64
# J = 128
interpolated_frames = interpolated_tensor[I, :, :, :]
print("interpolated_frames.shape:", interpolated_frames.shape)
# 表示したいフレームの範囲を設定
start_index = 60
end_index = 84
frames_to_display = interpolated_frames[start_index:end_index + 1]

num_frames = len(frames_to_display)
cols = 5  # 列数を設定（必要に応じて変更可能）
rows = math.ceil(num_frames / cols)  # 行数を自動計算

# サブプロットを作成
fig, axs = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))

# サブプロットの軸を1次元にフラット化してループ処理
for idx, ax in enumerate(axs.flat):
    if idx < num_frames:
        frame = frames_to_display[idx]
        img = ax.imshow(frame, cmap='gray')
        plt.colorbar(img, ax=ax)
        ax.set_title(f'Frame {start_index + idx}')
    else:
        ax.axis('off')  # 使用しないサブプロットを非表示に

In [ ]:
import imageio
import io

def create_heatmap_gif(data, output_path, duration=0.5):
    """
    m×n×nのnumpy配列をヒートマップのGIFに変換する。

    Parameters:
    - data: ndarray, shape (m, n, n)
    - output_path: str, 保存するGIFのパス
    - duration: float, 各フレームの表示時間（秒）
    """
    # データが3次元か確認
    if data.ndim != 3:
        raise ValueError("入力データは3次元のnumpy配列である必要があります。")

    m, n1, n2 = data.shape
    if n1 != n2:
        raise ValueError("各スライスはn×nの形状である必要があります。")

    # テンソルの最大値と最小値を計算
    vmin = np.min(data)
    vmax = np.max(data)

    images = []  # GIFに追加する画像のリスト

    # 図と軸を設定
    fig, ax = plt.subplots(figsize=(6, 6))
    # plt.axis('off')  # 軸を非表示

    # ヒートマップを初期化
    cax = ax.imshow(data[0], cmap='gray', vmin=vmin, vmax=vmax)
    # カラーバーを一度だけ追加
    cbar = fig.colorbar(cax, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=8)
    title = ax.set_title(f'Frame 1/{m}', fontsize=10)

    for i in range(m):
        # ヒートマップのデータを更新
        cax.set_data(data[i])
        # タイトルを更新
        title.set_text(f'Frame {i+1}/{m}')

        # レイアウトを固定
        fig.tight_layout()

        # 画像をバッファに保存
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.v2.imread(buf)
        images.append(image)
        buf.close()

    plt.close(fig)  # 図を閉じる

    # GIFとして保存
    imageio.mimsave(output_path, images, duration=duration)
    print(f"GIFが保存されました: {output_path}")

create_heatmap_gif(interpolated_frames, f"{DATA_PATH}/{EXP_DATE}/interpolated_i{I}_video.gif", duration=0.2)